In [16]:
import pycuda.autoinit
import numpy as np
import pycuda.gpuarray as gpuarray
import pycuda.driver as drv
from pycuda.reduction import ReductionKernel

# Reduction Kernels for summing elements, summing squares, and summing products.
sum_kernel = ReductionKernel(np.float64, neutral="0",
                             reduce_expr="a+b", map_expr="x[i]",
                             arguments="const double *x")

sum_sq_kernel = ReductionKernel(np.float64, neutral="0",
                                reduce_expr="a+b", map_expr="x[i]*x[i]",
                                arguments="const double *x")

product_sum_kernel = ReductionKernel(np.float64, neutral="0",
                                     reduce_expr="a+b", map_expr="x[i]*y[i]",
                                     arguments="const double *x, const double *y")


# Define a function to calculate Pearson's correlation coefficient
def pearson_correlation(x, y):
    n = x.size  # Use the size of the array instead of passing N
    
    x_gpu = gpuarray.to_gpu(np.asarray(x, np.float64))
    y_gpu = gpuarray.to_gpu(np.asarray(y, np.float64))
    
    # Perform the reduction operations on the GPU and fetch the results to the host.
    sum_x = sum_kernel(x_gpu).get()
    sum_y = sum_kernel(y_gpu).get()
    sum_x_sq = sum_sq_kernel(x_gpu).get()
    sum_y_sq = sum_sq_kernel(y_gpu).get()
    psum = product_sum_kernel(x_gpu, y_gpu).get()
    
    # Calculate the numerator and denominator for Pearson's correlation coefficient.
    num = psum - (sum_x * sum_y / n)
    den = np.sqrt((sum_x_sq - (sum_x ** 2 / n)) * (sum_y_sq - (sum_y ** 2 / n)))
    
    # Return the Pearson correlation coefficient.
    return num / den if den != 0 else 0


N = 50000000
# Generate random samples for tobacco, cancer, and infertility datasets
np.random.seed(0)  # Seed for reproducibility
tobacco_use = np.random.rand(N)
cancer_cases = np.random.rand(N)
infertility_cases = np.random.rand(N)

# Calculate Pearson's correlation coefficient for tobacco and cancer
correlation_tobacco_cancer = pearson_correlation(tobacco_use, cancer_cases)

# Calculate Pearson's correlation coefficient for tobacco and infertility
correlation_tobacco_infertility = pearson_correlation(tobacco_use, infertility_cases)

correlation_tobacco_cancer, correlation_tobacco_infertility


(-0.00015950359435246047, -8.318557123683293e-05)